In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [5]:
dataset = pd.read_csv('Train.csv')
dataset['Age'].fillna((dataset['Age'].mean()), inplace=True)
dataset['Embarked'].fillna('S', inplace=True)
X = dataset.iloc[:, [2,4,5,6,7,9,11]].values
y = dataset.iloc[:, 1].values

dataset2 = pd.read_csv('Test.csv')
dataset2['Age'].fillna((dataset['Age'].mean()), inplace=True)
dataset2['Embarked'].fillna('S', inplace=True)
X2 = dataset2.iloc[:, [1,3,4,5,6,8,10]].values

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 1] = le.fit_transform(X[:, 1])
X2[:, 1] = le.transform(X2[:, 1])
#1 Male, 0 Female
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6])], remainder='passthrough') 
X = np.array(ct.fit_transform(X)) 
X2 = np.array(ct.transform(X2))
#first 4 elements are the Class encoding
#S-> 0.0 0.0 1.0 0.0
#C-> 1.0 0.0 0.0 0.0
#Q-> 0.0 1.0 0.0 0.0

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [8]:
#feature scaling for NN. 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) #fit for the Train 
X_test = sc.transform(X_test) #just transform for Test
X_test2 = sc.transform(X2)

In [9]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [11]:
y_pred = classifier.predict(X_test2)
y_pred = (y_pred > 0.5) #works for each row was 0.5 changed to 0.68 for better results
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
# from sklearn.metrics import confusion_matrix, accuracy_score
# cm = confusion_matrix(y_test, y_pred)
# print(cm)
# accuracy_score(y_test, y_pred)
print(len(y_pred))
print(len(dataset2['PassengerId']))
first_col = dataset2.iloc[:,0].values
second_col = y_pred
output = np.concatenate((first_col.reshape(len(first_col),1), second_col.reshape(len(second_col),1)),1)
df = pd.DataFrame(output, columns = ['PassengerId','Survived'])
df.to_csv('Output.csv', index = False)

418
418
